<a href="https://colab.research.google.com/github/RummanAli/FYP2022-KnowledgeVision/blob/main/RESNET%2BGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Input, Add, Reshape
from tensorflow.keras.models import Model
import numpy as np
import keras
from keras import layers

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
x_train = x_train.astype(float)

In [ ]:
x_train  = np.expand_dims(x_train, axis=-1)

In [ ]:
x_train.shape

(60000, 28, 28, 1)

In [ ]:
input = Input(x_train.shape[1:])
f1 = Flatten()(input)
d1 = Dense(32, activation='relu')(f1)
d2 = Dense(64, activation='relu')(d1)
d3 = Dense(32, activation='relu')(d2)
added = Add()([d1, d3])
out = Dense(10, activation='softmax')(added)
model = Model(inputs=input, outputs=out)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
flatten_4 (Flatten)             (None, 784)          0           input_4[0][0]                    
__________________________________________________________________________________________________
dense_16 (Dense)                (None, 32)           25120       flatten_4[0][0]                  
__________________________________________________________________________________________________
dense_17 (Dense)                (None, 64)           2112        dense_16[0][0]                   
______________________________________________________________________________________________

In [ ]:
model.fit(x = x_train,y = y_train,epochs = 200,validation_data=(x_test,y_test))

Epoch 1/200
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2438 - accuracy: 0.9431 - val_loss: 0.4038 - val_accuracy: 0.9268
Epoch 2/200
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2568 - accuracy: 0.9387 - val_loss: 0.4025 - val_accuracy: 0.9286
Epoch 3/200
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2492 - accuracy: 0.9427 - val_loss: 0.3994 - val_accuracy: 0.9301
Epoch 4/200
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2437 - accuracy: 0.9434 - val_loss: 0.3948 - val_accuracy: 0.9317
Epoch 5/200
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2561 - accuracy: 0.9415 - val_loss: 0.4158 - val_accuracy: 0.9270
Epoch 6/200
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2559 - accuracy: 0.9401 - val_loss: 0.4165 - val_accuracy: 0.9307
Epoch 7/200
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2568 - accuracy: 0.9408 - val_loss: 0.4299 - val_ac

In [ ]:
#Discriminator
img = Input((28,28,1))
f1    = Flatten()(img)
d1    = Dense(32, activation='LeakyReLU')(f1)
d2    = Dense(64, activation='LeakyReLU')(d1)
d3    = Dense(32, activation='LeakyReLU')(d2)
dis   = Dense(1, activation='sigmoid')(d3)
dis = Model(inputs=img, outputs=dis)

#Generator
noise = Input((100))
d1    = Dense(1000, activation='tanh')(noise)
d2    = Dense(1000, activation='tanh')(d1)
out   = Dense(784 , activation='tanh')(d2)
gen   = Reshape((28,28,1))(out)
gen = Model(inputs = noise, outputs = gen)

In [ ]:
true = np.ones(x_train.shape[0])
false = np.zeros(x_train.shape[0])
random_latent_vectors = tf.random.normal(shape=(32, 100))
generated_imgs = gen((random_latent_vectors))

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(32, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        misleading_labels = tf.zeros((batch_size, 1))
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }


In [ ]:
epochs = 30  # In practice, use ~100 epochs

gan = GAN(discriminator=dis, generator=gen, latent_dim=100)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

gan.fit(
    x_train,epochs=epochs
)

Epoch 1/30
1875/1875 [==============================] - 44s 23ms/step - d_loss: 0.1304 - g_loss: 2.2774
Epoch 2/30
1875/1875 [==============================] - 43s 23ms/step - d_loss: 0.0047 - g_loss: 5.9326
Epoch 3/30
1875/1875 [==============================] - 43s 23ms/step - d_loss: -0.0899 - g_loss: 13.9054
Epoch 4/30
1875/1875 [==============================] - 43s 23ms/step - d_loss: -0.3244 - g_loss: 34.1198
Epoch 5/30
1875/1875 [==============================] - 43s 23ms/step - d_loss: -0.5505 - g_loss: 53.6335
Epoch 6/30
1875/1875 [==============================] - 43s 23ms/step - d_loss: -0.6272 - g_loss: 61.3128
Epoch 7/30
1875/1875 [==============================] - 43s 23ms/step - d_loss: -1.1504 - g_loss: 106.3172
Epoch 8/30
1875/1875 [==============================] - 44s 23ms/step - d_loss: -2.6167 - g_loss: 231.0446
Epoch 9/30
1875/1875 [==============================] - 44s 24ms/step - d_loss: -6.2634 - g_loss: 537.7417
Epoch 10/30
1875/1875 [========================

In [ ]:
gen.save('/content')

INFO:tensorflow:Assets written to: /content/assets


In [ ]:
gen(noise)